In [1]:
import time
from selenium import webdriver
from datetime import datetime
import os
from bs4 import BeautifulSoup
import pandas as pd

# Webscraping of fbref.com for football stats

In [60]:
# Path to the chromedriver.exe
WEBDRIVER_PATH = os.getcwd() + '\\chromedriver.exe'

In [61]:
# Opening the driver
driver = webdriver.Chrome(executable_path=WEBDRIVER_PATH)

# Url of premier league stats
url = 'https://fbref.com/en/comps/9/stats/Premier-League-Stats'
driver.get(url)

<ipython-input-61-1ee50004c406>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=WEBDRIVER_PATH)


In [62]:
# Getting the table
content = driver.find_element_by_css_selector('#div_stats_standard') 

# Making soup
soup = BeautifulSoup(content.get_attribute('innerHTML'))

<ipython-input-62-d4e1698a6ea5>:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  content = driver.find_element_by_css_selector('#div_stats_standard')


## All the functions used

In [114]:
def init_selenium(chrome_path):
    # Opening the driver
    driver = webdriver.Chrome(executable_path=WEBDRIVER_PATH)
    return driver

def webscraper(url, chrome_path, league_name, year_stop):
    """
    Enter in the url and this will click around and scrape the data on fbref.com for the particular league chosen.  
    
    Input:
        url             : Url for the current season webpage to start extracting from
        chrome_path     : path to the chromium webdriver
        league_name     : Name of the league the url is scraping from. This creates folder and filenames
        year_stop       : The last season the webscraper goes to (Premier league goes to 92, Ligue 1 goes to 95(?))
    """
    
    # Starting webdriver
    driver = init_selenium(chrome_path)
    driver.get(url)
    
    # Click "disagree cookies" - button
    disagree = driver.find_element_by_css_selector("#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button:nth-child(1)")
    disagree.click()
    print("Disagree with cookies.")
    
    
    # Emulate human behaviour
    time.sleep(random.randint(0, 3))

    
    # Looping through all the seasons for that league
    while True:
        
        # Get seasons
        info = driver.find_element_by_css_selector('#info')
        info_soup = BeautifulSoup(info.get_attribute('innerHTML'))
        season = info_soup.find('h1').text.split()[0]
        print("Fetching data for season " + season)

        
        # Getting the table under #div_stats_standard
        content = driver.find_element_by_css_selector('#div_stats_standard') 
        soup = BeautifulSoup(content.get_attribute('innerHTML'))

        # Extract table and tbody (table body)
        table = soup.find('table')
        tbody = table.find('tbody')

        # Appending every cell of data to a list
        data = []
        for tr_body in tbody.find_all('tr'):
            row = []
            for td in tr_body.find_all('td'):
                row.append(td.get_text())
            data.append(row)
        
        # Saving the table to a dataframe
        df = pd.DataFrame(data)

        # Get header
        header = table.find('thead').find_all('tr')[1].get_text().split('\n')[2:-1]
        df.columns=header
        df = df.dropna() # Dropping Nan rows
        df["Season"] = season 

        # Saving each season as a separate csv
        save_data(df, season, league_name)
        
        
        # Setting 
        if "92" in season:
            print("Last season found, 1992/1993")
            return 
        
        
        # Go to next page
        try:
            # For the first "previous button"
            prev_button = driver.find_element_by_css_selector("#meta > div:nth-child(2) > div > a")
            prev_button.click()
            print("Going to next season")
        except:
            print("Did not find click button")
            
        
        # Emulate human behaviour
        time.sleep(random.randint(0, 3))

    return 

def save_data(df,season, league_name):
    data_path = os.path.join(os.getcwd(), 'data')
    
    # Checking if data path exists or not
    if os.path.exists(data_path) == False:
        os.mkdir(data_path)
    
    league_folder = os.path.join(data_path, league_name)
    
    if os.path.exists(league_folder) == False:
        print("Creating league folder")
        os.mkdir(league_folder)
    else:
        print("Folder for "+  league_name + " found.")
        
    # Saving the dataframe as csv
    file_name = league_name + "_" + season + ".csv"
    file_path = os.path.join(data_path, file_name)
    print("Saving data to: \n" + file_path)

    df.to_csv(file_path)
    

In [119]:
# Initializing inputs

chrome_path = os.getcwd() + '\\chromedriver.exe'

url = 'https://fbref.com/en/comps/12/10731/stats/2020-2021-La-Liga-Stats'

league_name = "La Liga"

year_stop = "95"


webscraper(url, chrome_path, "La Liga", "95")



<ipython-input-114-59a948df01a6>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=WEBDRIVER_PATH)
<ipython-input-114-59a948df01a6>:16: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  disagree = driver.find_element_by_css_selector("#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button:nth-child(1)")


Disagree with cookies.


<ipython-input-114-59a948df01a6>:24: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  info = driver.find_element_by_css_selector('#info')
<ipython-input-114-59a948df01a6>:32: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  content = driver.find_element_by_css_selector('#div_stats_standard')


Fetching data for season 2020-2021
Saving data to: 
C:\Users\hakon\Documents\GitHub\DataSciencePortfolio\ExpectedTransferFee\WebScraping\data\La Liga_2020-2021.csv


<ipython-input-114-59a948df01a6>:68: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  prev_button = driver.find_element_by_css_selector("#meta > div:nth-child(2) > div > a")


...

<ipython-input-114-59a948df01a6>:68: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  prev_button = driver.find_element_by_css_selector("#meta > div:nth-child(2) > div > a")


Going to next season


<ipython-input-114-59a948df01a6>:24: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  info = driver.find_element_by_css_selector('#info')
<ipython-input-114-59a948df01a6>:32: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  content = driver.find_element_by_css_selector('#div_stats_standard')


Fetching data for season 1992-1993
Saving data to: 
C:\Users\hakon\Documents\GitHub\DataSciencePortfolio\ExpectedTransferFee\WebScraping\data\La Liga_1992-1993.csv
Last season found, 1992/1993


NameError: name 'final_df' is not defined

In [5]:
# Extract table
table = soup.find('table')

# Get body
tbody = table.find('tbody')


In [8]:
# Extract table
table = soup.find('table')

# Appending every cell of data to a list
data = []
for tr_body in tbody.find_all('tr'):
    row = []
    for td in tr_body.find_all('td'):
        row.append(td.get_text())
    data.append(row)

In [9]:
df = pd.DataFrame(data)

# Get header
header = table.find('thead').find_all('tr')[1].get_text().split('\n')[2:-1]
df.columns=header
df = df.dropna() # Dropping Nan rows
df.loc[24:27] # Showing that the NaN rows are dropped

Player   Nation Pos           Squad     Age  Born  MP  \
24  Pierre-Emerick Aubameyang   ga GAB  FW         Arsenal  32-305  1989  14   
26                Jordan Ayew   gh GHA  FW  Crystal Palace  30-220  1991  27   
27                Luke Ayling  eng ENG  DF    Leeds United  30-237  1991  23   

   Starts    Min   90s  ...   xG npxG   xA npxG+xA    xG    xA xG+xA  npxG  \
24     12  1,037  11.5  ...  5.8  4.1  0.8     5.0  0.50  0.07  0.57  0.36   
26     20  1,722  19.1  ...  2.9  2.9  2.2     5.2  0.15  0.12  0.27  0.15   
27     23  2,066  23.0  ...  1.5  1.5  0.8     2.3  0.07  0.03  0.10  0.07   

   npxG+xA  Matches  
24    0.43  Matches  
26    0.27  Matches  
27    0.10  Matches  

[3 rows x 32 columns]